<a href="https://colab.research.google.com/github/eghib22/Store-Sales-Forecasting/blob/main/model_experiment_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p ~/.kaggle
from google.colab import files
files.upload()
!mv "kaggle.json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!ls -l ~/.kaggle/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving kaggle.json to kaggle.json
total 4
-rw------- 1 root root 71 Jul 13 17:48 kaggle.json


In [2]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -o walmart-recruiting-store-sales-forecasting
!unzip -o '*.csv.zip'


walmart-recruiting-store-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  features.csv.zip
  inflating: features.csv            

Archive:  test.csv.zip
  inflating: test.csv                

4 archives were successfully processed.


In [3]:
!pip install prophet
!pip install -q dagshub mlflow

import dagshub
dagshub.init(repo_owner='eghib22', repo_name='Store-Sales-Forecasting', mlflow=True)


Accessing as agasi22

Initialized MLflow to track repo "eghib22/Store-Sales-Forecasting"

Repository eghib22/Store-Sales-Forecasting initialized!

In [4]:
import mlflow
import logging
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
logging.getLogger("prophet").setLevel(logging.WARNING)
logging.basicConfig(level=logging.WARNING)

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import gc


In [5]:
train = pd.read_csv('train.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')

train['Date'] = pd.to_datetime(train['Date'])
features['Date'] = pd.to_datetime(features['Date'])

df = train.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
df = df.merge(stores, on='Store', how='left')
df = df.sort_values('Date')


In [6]:
for col in ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']:
    df[col] = df[col].fillna(method='ffill').fillna(method='bfill')

correlations = df[['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']].corr()
print(correlations)

# Drop regressors with low absolute correlation < 0.05
use_regressors = []
for reg in ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']:
    if abs(correlations.loc['Weekly_Sales', reg]) >= 0.05:
        use_regressors.append(reg)

print(f"Using regressors: {use_regressors}")


              Weekly_Sales  Temperature  Fuel_Price       CPI  Unemployment
Weekly_Sales      1.000000    -0.002312   -0.000120 -0.020921     -0.025864
Temperature      -0.002312     1.000000    0.143859  0.182112      0.096730
Fuel_Price       -0.000120     0.143859    1.000000 -0.164210     -0.033853
CPI              -0.020921     0.182112   -0.164210  1.000000     -0.299953
Unemployment     -0.025864     0.096730   -0.033853 -0.299953      1.000000
Using regressors: []


In [7]:
def weighted_mae(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)


In [8]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error

results = []
all_preds = []

store_dept_groups = df.groupby(['Store', 'Dept'])
total_groups = len(store_dept_groups)

print(f"--- Starting Prophet Forecasting for {total_groups} Store-Dept combos ---")

mlflow.set_experiment("Prophet_Forecast")

from prophet.make_holidays import make_holidays_df
us_holidays = make_holidays_df(year_list=[2010, 2011, 2012], country='US')

with mlflow.start_run(run_name="Prophet_Improved_FE_Tuning"):
    mlflow.log_param("model", "Prophet")
    mlflow.log_param("seasonality_yearly", True)
    mlflow.log_param("seasonality_weekly", True)
    mlflow.log_param("seasonality_daily", False)
    mlflow.log_param("seasonality_mode", "multiplicative")
    mlflow.log_param("changepoint_prior_scale", 0.3)
    mlflow.log_param("extra_regressors", use_regressors)
    mlflow.log_param("holidays", "US")

    for idx, ((store_id, dept_id), group) in enumerate(store_dept_groups, start=1):
        print(f"\n--- Processing Store: {store_id}, Dept: {dept_id} ({idx}/{total_groups}) ---")

        g = group.sort_values('Date').copy()
        g['ds'] = g['Date']
        g['y'] = g['Weekly_Sales']
        weights = g['IsHoliday'].apply(lambda x: 5 if x else 1)

        y_train = g[g['ds'] < '2012-01-01']
        y_val = g[(g['ds'] >= '2012-01-01') & (g['ds'] < '2012-07-01')]
        weights_val = weights.loc[y_val.index]

        if len(y_train) < 100 or len(y_val) < 20:
            print(f"   Skipped: Not enough data (Train: {len(y_train)}, Val: {len(y_val)})")
            continue

        try:
            model = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=False,
                changepoint_prior_scale=0.3,
                seasonality_mode='multiplicative',
                holidays=us_holidays
            )

            for reg in use_regressors:
                model.add_regressor(reg)

            model.fit(y_train[['ds', 'y'] + use_regressors])

            future = y_val[['ds'] + use_regressors].copy()
            forecast = model.predict(future)

            y_pred = forecast['yhat'].values
            y_true = y_val['y'].values

            wmae = weighted_mae(y_true, y_pred, weights_val)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))

            print(f"   WMAE: {wmae:.2f} | RMSE: {rmse:.2f}")

            results.append({
                'Store': store_id,
                'Dept': dept_id,
                'RMSE': rmse,
                'WMAE': wmae
            })

            all_preds.append(pd.DataFrame({
                'Date': y_val['ds'].values,
                'Store': store_id,
                'Dept': dept_id,
                'y_true': y_true,
                'y_pred': y_pred,
                'weight': weights_val.values
            }))

        except Exception as e:
            print(f"   Failed for Store {store_id}, Dept {dept_id}: {str(e)}")
            continue

        gc.collect()

    print("\n--- Prophet Forecasting Complete ---")

    if len(all_preds) > 0:
        all_df = pd.concat(all_preds)
        overall_wmae = np.sum(all_df['weight'] * np.abs(all_df['y_true'] - all_df['y_pred'])) / np.sum(all_df['weight'])
        print(f"\n Overall WMAE: {overall_wmae:.2f}")

        mlflow.log_metric("Overall_WMAE", overall_wmae)
        mlflow.log_metric("Total_StoreDept_Models", len(results))
    else:
        print("\nNo valid predictions to aggregate!")

    results_df = pd.DataFrame(results)
    print("\n--- Results Head ---")
    print(results_df.head())

    results_df.to_csv('/content/drive/MyDrive/prophet_improved_results.csv', index=False)

print("\n Done — results saved & run logged to MLflow.")
mlflow.end_run()


--- Starting Prophet Forecasting for 3331 Store-Dept combos ---

--- Processing Store: 1, Dept: 1 (1/3331) ---


DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pcs8oqd/zfza3f9m.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pcs8oqd/4nwn2gvy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=14029', 'data', 'file=/tmp/tmp9pcs8oqd/zfza3f9m.json', 'init=/tmp/tmp9pcs8oqd/4nwn2gvy.json', 'output', 'file=/tmp/tmp9pcs8oqd/prophet_model8778628_/prophet_model-20250713174911.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:49:11 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:49:11 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


   WMAE: 4456.63 | RMSE: 7423.06


DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pcs8oqd/jrqh21x3.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pcs8oqd/po203kn_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=49334', 'data', 'file=/tmp/tmp9pcs8oqd/jrqh21x3.json', 'init=/tmp/tmp9pcs8oqd/po203kn_.json', 'output', 'file=/tmp/tmp9pcs8oqd/prophet_modelil3o5ou9/prophet_model-20250713174911.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:49:11 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:49:11 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



--- Processing Store: 1, Dept: 2 (2/3331) ---
   WMAE: 3550.27 | RMSE: 4526.63


...

DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pcs8oqd/9mcsa5ge.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pcs8oqd/75lmekrz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=38975', 'data', 'file=/tmp/tmp9pcs8oqd/9mcsa5ge.json', 'init=/tmp/tmp9pcs8oqd/75lmekrz.json', 'output', 'file=/tmp/tmp9pcs8oqd/prophet_modelro1p7tcx/prophet_model-20250713180640.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:06:40 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:06:40 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



--- Processing Store: 45, Dept: 94 (3327/3331) ---
   Skipped: Not enough data (Train: 91, Val: 26)

--- Processing Store: 45, Dept: 95 (3328/3331) ---


DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pcs8oqd/q_gya034.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pcs8oqd/32mj6psx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=98827', 'data', 'file=/tmp/tmp9pcs8oqd/q_gya034.json', 'init=/tmp/tmp9pcs8oqd/32mj6psx.json', 'output', 'file=/tmp/tmp9pcs8oqd/prophet_modelr4rbdl5t/prophet_model-20250713180640.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:06:40 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:06:40 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


   WMAE: 5763.91 | RMSE: 6763.76

--- Processing Store: 45, Dept: 96 (3329/3331) ---
   Skipped: Not enough data (Train: 0, Val: 1)

--- Processing Store: 45, Dept: 97 (3330/3331) ---
   WMAE: 575.60 | RMSE: 689.73

--- Processing Store: 45, Dept: 98 (3331/3331) ---
   Skipped: Not enough data (Train: 92, Val: 26)

--- Prophet Forecasting Complete ---



 Overall WMAE: 2389.95

--- Results Head ---
   Store  Dept         RMSE         WMAE
0      1     1  7423.062232  4456.629273
1      1     2  4526.625240  3550.272698
2      1     3  1394.364730  1186.401674
3      1     4  2269.679844  1589.133981
4      1     5  4280.572445  2950.644652
🏃 View run Prophet_Improved_FE_Tuning at: https://dagshub.com/eghib22/Store-Sales-Forecasting.mlflow/#/experiments/7/runs/6c38cdb6b4314b03a3c71467e6aa7959
🧪 View experiment at: https://dagshub.com/eghib22/Store-Sales-Forecasting.mlflow/#/experiments/7

 Done — results saved & run logged to MLflow.
